In [1]:
!git clone https://github.com/stefan-it/nmt-mk-en.git

Cloning into 'nmt-mk-en'...
remote: Enumerating objects: 64, done.
remote: Total 64 (delta 0), reused 0 (delta 0), pack-reused 64
Unpacking objects: 100% (64/64), done.


In [2]:
%cd nmt-kn-te/

/content/nmt-kn-ml


In [4]:
! chmod a+x /content/nmt-kn-te/scripts/data_preparation.sh

In [5]:
! /content/nmt-kn-te/scripts/data_preparation.sh data.kn data.te

Cloning into 'mosesdecoder'...
remote: Enumerating objects: 148090, done.
remote: Counting objects: 100% (518/518), done.
remote: Compressing objects: 100% (223/223), done.
remote: Total 148090 (delta 319), reused 443 (delta 292), pack-reused 147572
Receiving objects: 100% (148090/148090), 129.87 MiB | 19.93 MiB/s, done.
Resolving deltas: 100% (114345/114345), done.
Cloning into 'subword-nmt'...
remote: Enumerating objects: 587, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 587 (delta 1), reused 4 (delta 1), pack-reused 576
Receiving objects: 100% (587/587), 244.20 KiB | 2.60 MiB/s, done.
Resolving deltas: 100% (350/350), done.
Cloning into 'seq2seq'...
remote: Enumerating objects: 5995, done.
remote: Total 5995 (delta 0), reused 0 (delta 0), pack-reused 5995
Receiving objects: 100% (5995/5995), 1.63 MiB | 10.39 MiB/s, done.
Resolving deltas: 100% (4189/4189), done.
Tokenizer Version 1.1
Language: sk
Number of threads

In [6]:
! chmod a+x /content/nmt-kn-te/scripts/split_dataset.sh

In [7]:
! /content/nmt-kn-te/scripts/split_dataset.sh corpus.clean.bpe.32000.kn corpus.clean.bpe.32000.te

In [8]:
%cd ..

/content


In [9]:
%%capture
 # w and b (wandb) for logging
! pip install wandb

# sacremos - for tokenizing
! pip install sacremos

# fairseq - for training and evaluation of the model
! git clone https://github.com/pytorch/fairseq
%cd fairseq
! pip install --editable ./
%cd ..

%cd /content/fairseq/
!python -m pip install --editable .
%cd /content

! echo $PYTHONPATH

import os
os.environ['PYTHONPATH'] += ":/content/fairseq/"

! echo $PYTHONPATH

In [10]:
!fairseq-preprocess --source-lang kn --target-lang te \
--trainpref nmt-kn-te/train/train \
--validpref nmt-kn-te/dev/dev \
--testpref nmt-kn-te/test/test \
--destdir nmt-kn-te/tokenized.kn-te \
--thresholdsrc 2 \
--thresholdtgt 2 

2022-03-05 03:35:54 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-03-05 03:35:54 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='nmt-kn-te/tokenized.kn-te', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=False, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=False, quantization_co

In [11]:
import wandb
wandb.login()
# login authorization.

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [12]:
!fairseq-train /content/nmt-kn-te/tokenized.kn-te \
--arch lstm \
  --dropout 0.2 \
  --optimizer adam --lr 0.005 --lr-shrink 0.5 \
  --max-tokens 12000 \
  --wandb-project "Simple LSTM - kn to te - 5 March 2022"

2022-03-05 03:37:40 | INFO | numexpr.utils | NumExpr defaulting to 2 threads.
2022-03-05 03:37:42 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-03-05 03:37:44 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'tensorboard_logdir': None, 'wandb_project': 'Simple LSTM - kn to te - 5 March 2022', 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1,

KeyboardInterrupt: ignored

In [13]:
!pwd

/content


In [14]:
!fairseq-generate /content/nmt-kn-te/tokenized.kn-te \
  --path checkpoints/checkpoint_best.pt \
  --beam 5 \
  --remove-bpe

Streaming output truncated to the last 5000 lines.
D-919	-0.576632559299469	లైఫ ్ స ్ పాన ్ స ్
P-919	-0.8071 -0.0002 -0.2239 -0.0001 -2.2628 -0.0002 -0.0823 -0.0001 -1.8131
S-731	ಯಾರೂ ಬರೆಯುತ ್ ತಾರೆ ನಾನು ಬರೆಯುತ ್ ತೇನೆ ಇದು ಬೇಡ .
T-731	నాకు ఎవరూ రాయలేదు కూడా .
H-731	-1.1884760856628418	ఎవరూ రాస ్ తున ్ నాను .
D-731	-1.1884760856628418	ఎవరూ రాస ్ తున ్ నాను .
P-731	-2.2140 -3.0490 -0.0668 -3.6473 -0.0018 -0.4648 -0.0567 -0.0073
S-776	&quot; &quot; &quot; ಕಮ ್ ಆನ ್ ಡೌನ ್ ! &quot;
T-776	“ లోపలికి రావచ ్ చా !
H-776	-1.1197210550308228	&quot; &quot; &quot; అన ్ నమాట ! &quot;
D-776	-1.1197210550308228	&quot; &quot; &quot; అన ్ నమాట ! &quot;
P-776	-1.6533 -0.1070 -0.0038 -3.9557 -0.0004 -3.7017 -0.5136 -0.1342 -0.0079
S-852	ಸ ್ ವಲ ್ ಪ ಭಯವೂ ಆಗುತ ್ ತಿದೆ .
T-852	కాస ్ త భయం పోతుంది .
H-852	-0.9124902486801147	ఈరోజు కొంచెం ఒడిదుడుకులు ఉంటాయి .
D-852	-0.9124902486801147	ఈరోజు కొంచెం ఒడిదుడుకులు ఉంటాయి .
P-852	-3.7966 -1.0796 -0.5394 -0.0552 -0.0028 -0.0013
S-91	ಆಕೆಯ ಮಾತುಗಳು ಸ ್ ಪಷ ್ ಟವಾಗಿರಲಿಲ ್ ಲ .
